# Standard GP

In [1]:
import fastgps
import qmcpy as qp
import torch
import numpy as np

In [2]:
torch.set_default_dtype(torch.float64)

## True Function

In [3]:
def f_ackley(x, a=20, b=0.2, c=2*np.pi, scaling=32.768):
    # https://www.sfu.ca/~ssurjano/ackley.html
    assert x.ndim==2
    x = 2*scaling*x-scaling
    t1 = a*torch.exp(-b*torch.sqrt(torch.mean(x**2,1)))
    t2 = torch.exp(torch.mean(torch.cos(c*x),1))
    t3 = a+np.exp(1)
    y = -t1-t2+t3
    return y
d = 1 # dimension
rng = torch.Generator().manual_seed(17)
x = torch.rand((2**7,d),generator=rng) # random testing locations
y = f_ackley(x) # true values at random testing locations
z = torch.rand((2**8,d),generator=rng) # other random locations at which to evaluate covariance
print("x.shape = %s"%str(tuple(x.shape)))
print("y.shape = %s"%str(tuple(y.shape)))
print("z.shape = %s"%str(tuple(z.shape)))

x.shape = (128, 1)
y.shape = (128,)
z.shape = (256, 1)


## Construct GP

In [4]:
fgp = fastgps.StandardGP(qp.KernelGaussian(d,torchify=True),seqs=7)
x_next = fgp.get_x_next(2**6)
y_next = f_ackley(x_next)
fgp.add_y_next(y_next)
print("x_next.shape = %s"%str(tuple(x_next.shape)))
print("y_next.shape = %s"%str(tuple(y_next.shape)))

x_next.shape = (64, 1)
y_next.shape = (64,)


In [5]:
pmean = fgp.post_mean(x)
print("pmean.shape = %s"%str(tuple(pmean.shape)))
print("l2 relative error = %.2e"%(torch.linalg.norm(y-pmean)/torch.linalg.norm(y)))

pmean.shape = (128,)
l2 relative error = 1.67e-01


In [6]:
data = fgp.fit()
list(data.keys())

     iter of 5.0e+03 | best loss  | loss       | term1      | term2     
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            0.00e+00 | 4.13e+06   | 4.13e+06   | 8.27e+06   | -5.54e+02 
            5.00e+00 | 2.41e+06   | 2.41e+06   | 4.81e+06   | -5.35e+02 
            1.00e+01 | 1.94e+05   | 1.94e+05   | 3.89e+05   | -3.53e+02 
            1.50e+01 | 2.40e+02   | 2.49e+02   | 3.15e+01   | 3.48e+02  
            2.00e+01 | 2.09e+02   | 2.11e+02   | 4.02e+01   | 2.64e+02  
            2.50e+01 | 2.05e+02   | 2.07e+02   | 7.10e+01   | 2.25e+02  
            3.00e+01 | 2.05e+02   | 2.05e+02   | 5.88e+01   | 2.34e+02  
            3.50e+01 | 2.05e+02   | 2.05e+02   | 6.26e+01   | 2.30e+02  
            4.00e+01 | 2.05e+02   | 2.05e+02   | 6.35e+01   | 2.29e+02  
            4.50e+01 | 2.05e+02   | 2.05e+02   | 6.34e+01   | 2.29e+02  
            4.60e+01 | 2.05e+02   | 2.05e+02   | 6.38e+01   | 2.29e+02  


[]

In [9]:
pmean,pvar,q,ci_low,ci_high = fgp.post_ci(x,confidence=0.99)
print("pmean.shape = %s"%str(tuple(pmean.shape)))
print("pvar.shape = %s"%str(tuple(pvar.shape)))
print("q = %.2f"%q)
print("ci_low.shape = %s"%str(tuple(ci_low.shape)))
print("ci_high.shape = %s"%str(tuple(ci_high.shape)))
print("l2 relative error = %.2e"%(torch.linalg.norm(y-pmean)/torch.linalg.norm(y)))
pcov = fgp.post_cov(x,x)
print("pcov.shape = %s"%str(tuple(pcov.shape)))
assert torch.allclose(pcov.diagonal(),pvar) and (pvar>=0).all()
pcov2 = fgp.post_cov(x,z)
print("pcov2.shape = %s"%str(tuple(pcov2.shape)))

pmean.shape = (128,)
pvar.shape = (128,)
q = 2.58
ci_low.shape = (128,)
ci_high.shape = (128,)
l2 relative error = 7.31e-02
pcov.shape = (128, 128)
pcov2.shape = (128, 256)


In [10]:
pcmean,pcvar,q,cci_low,cci_high = fgp.post_cubature_ci(confidence=0.99)
print("pcmean = %.3e"%pcmean)
print("pcvar = %.3e"%pcvar)
print("cci_low = %.3e"%cci_low)
print("cci_high = %.3e"%cci_high)

pcmean = 1.830e+01
pcvar = 1.030e-03
cci_low = 1.822e+01
cci_high = 1.838e+01


## Project and Increase Sample Size

In [11]:
pcov_future = fgp.post_cov(x,z,n=2*fgp.n)
pvar_future = fgp.post_var(x,n=2*fgp.n)
pcvar_future = fgp.post_cubature_var(n=2*fgp.n)

In [12]:
x_next = fgp.get_x_next(2*fgp.n)
y_next = f_ackley(x_next)
fgp.add_y_next(y_next)
print("l2 relative error = %.2e"%(torch.linalg.norm(y-fgp.post_mean(x))/torch.linalg.norm(y)))
assert torch.allclose(fgp.post_cov(x,z),pcov_future)
assert torch.allclose(fgp.post_var(x),pvar_future)
assert torch.allclose(fgp.post_cubature_var(),pcvar_future)

l2 relative error = 6.18e-02


In [13]:
data = fgp.fit(verbose=False)
print("l2 relative error = %.2e"%(torch.linalg.norm(y-fgp.post_mean(x))/torch.linalg.norm(y)))

l2 relative error = 6.59e-02


In [14]:
pcov_16n = fgp.post_cov(x,z,n=16*fgp.n)
pvar_16n = fgp.post_var(x,n=16*fgp.n)
pcvar_16n = fgp.post_cubature_var(n=16*fgp.n)
x_next = fgp.get_x_next(16*fgp.n)
y_next = f_ackley(x_next)
fgp.add_y_next(y_next)
assert torch.allclose(fgp.post_cov(x,z),pcov_16n)
assert torch.allclose(fgp.post_var(x),pvar_16n)
assert torch.allclose(fgp.post_cubature_var(),pcvar_16n)